# Get Ford dealer locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [45]:
headers = {
    'authority': 'liveapi.yext.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    'origin': 'https://www.autozone.com',
    'referer': 'https://www.autozone.com/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'x-requested-by': 'MF',
}

params = {
    'location': '90066',
    'api_key': 'a427dc0cb3e4f080da0ebe74621b8020',
    'v': '20180731',
    'radius': '500',
    'filters': '[{"countryCode":{"includes":["US","PR","VI"]}}]',
    'offset':10
}

response = requests.get('https://liveapi.yext.com/v2/accounts/me/locations/geosearch', params=params, headers=headers)

In [46]:
count = response.json()['response']['count']
count

898

In [47]:
src = pd.DataFrame(response.json()['response']['locations'])

In [48]:
len(src)

10

In [49]:
src['address']

0         3102 Century Blvd
1       1905 Western Ave #2
2           1401 W Florence
3       2876 W Imperial Hwy
4      13765 Hawthorne Blvd
5        1306-13 Highland A
6        2907 S Vermont Ave
7     1250 S Vermont Ave #1
8    1325 W Washington Blvd
9    5852 Santa Monica Blvd
Name: address, dtype: object